In [1]:
!pip install git+https://github.com/h44z/pygitea

  Cloning https://github.com/h44z/pygitea to c:\users\user\appdata\local\temp\pip-req-build-hi3nea5b
  Created wheel for pygitea: filename=pygitea-0.0-cp37-none-any.whl size=4121 sha256=7dc5bd4762e4f5aa17f48ae1de0418451d26b93e73c9d3735e2db3c20d0b539b
  Stored in directory: C:\Users\User\AppData\Local\Temp\pip-ephem-wheel-cache-oy2jitr7\wheels\29\c9\b2\7e46bec0d76d405d6f586c1baf3e22e12b2b9393e785ed8aa4
Successfully built pygitea


  Running command git clone -q https://github.com/h44z/pygitea 'C:\Users\User\AppData\Local\Temp\pip-req-build-hi3nea5b'


In [1]:
import pygitea
import json
import requests
from pandas.io.json import json_normalize
from sqlalchemy import create_engine, engine, text, types, MetaData, Table, String
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%load_ext autotime
import numpy as np
from datetime import datetime
from sqlalchemy import String


In [2]:
import config
import os

time: 5.96 ms


In [3]:
def objects_to_strings(table):
    measurer = np.vectorize(len)
    df_object = table.select_dtypes(include=[object])
    string_columns = dict(zip(df_object, measurer(
        df_object.values.astype(str)).max(axis=0)))
    string_columns = {key: String(length=value) if value > 0 else String(length=1)
                      for key, value in string_columns.items() }
    return string_columns

time: 2 ms


In [4]:
engine = create_engine(config.SQL_ALCHEMY_STRING)

time: 78 ms


# Gitea APIs

In [ ]:
api = pygitea.API(config.GITEA_APP_URL, token=config.GITEA_TOKEN)

## 1. Organizations

In [6]:
organizations_list = []
for i in range(1,3):
    organizations_list = organizations_list + api.get('/admin/orgs?page={}&limit=50'.format(i)).json()
organizations_df = json_normalize(organizations_list)

time: 771 ms


In [7]:
organizations_df.to_sql(con=engine, name='organizations',
                 if_exists='replace', dtype=objects_to_strings(organizations_df))

time: 7.39 s


## 2. All Users

In [5]:
users = api.get('/admin/users')
users_df = json_normalize(users.json())

In [ ]:
users_df.to_sql(con=engine, name='users',
                 if_exists='replace', dtype=objects_to_strings(users_df))

## 3. All Repositories

In [6]:
repos = []
for index, row in organizations_df.iterrows():
    repos = repos + api.get('/orgs/{}/repos'.format(row['username'])).json()

for index, row in users_df.iterrows():
    repos = repos + api.get('/users/{}/repos'.format(row['login'])).json()

In [7]:
repos_df = json_normalize(repos)

In [458]:
repos_df.to_sql(con=engine, name='repos',
                 if_exists='replace', dtype=objects_to_strings(repos_df))

## 4. All Branches of All Repos

In [232]:
branches = []
for index, row in repos_df.iterrows():
    branches_temp = api.get('/repos/{}/{}/branches'.format(row['owner.login'],row['name'])).json()
    branches_temp = [dict(item, **{'repo_name':'{}'.format(row['name'])}) for item in branches_temp]    
    branches_temp = [dict(item, **{'owner':'{}'.format(row['owner.login'])}) for item in branches_temp] 
    branches = branches + branches_temp
branches_df = json_normalize(branches)

time: 29 s


In [233]:
branches_df.to_sql(con=engine, name='branches',
                 if_exists='replace', dtype=objects_to_strings(branches_df))

time: 2.03 s


C:\Users\User\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE2\\x80\\x8C' for column 'commit.message' at row 330")
  result = self._query(query)


## 5. Commits of all Repos

In [ ]:
# commits = []
# for index, row in repos_df.iterrows():
#     first_commit_pg = api.get('/repos/{}/{}/commits'.format(row['owner.login'],row['name']))
#     first_commit_pg_list = [dict(item, **{'repo_name':'{}'.format(row['name'])}) for item in first_commit_pg.json()]    
#     first_commit_pg_list = [dict(item, **{'repo_full_name':'{}'.format(row['full_name'])}) for item in first_commit_pg_list] 
#     nr_pages = int(first_commit_pg.headers['X-Pagecount'])
#     commits = commits + first_commit_pg_list
#     if nr_pages > 1:
#         for i in range(2, nr_pages+1):
#             commits = commits + api.get('/repos/{}/{}/commits/?page={}'.format(row['owner.login'],row['name'],i)).json()

## 6. All Commits of an Org

In [148]:
def repos_of_org(org):
    org_repos = repos_df[repos_df['owner.login'] == org]
    return org_repos

time: 3.02 ms


In [179]:
def get_commits_per_page(owner, repo, page, api):
    commit_pg = api.get('/repos/{}/{}/commits/?page={}'.format(owner, repo, page))
    commit_pg_list = [dict(item, **{'repo_name':'{}'.format(repo)}) for item in commit_pg.json()]    
    commit_pg_list = [dict(item, **{'owner':'{}'.format(owner)}) for item in commit_pg_list]
    return commit_pg, commit_pg_list

time: 998 µs


In [174]:
def commits_of_repo(repo, owner, api):
    first_commit_pg, first_commit_pg_list = get_commits_per_page(owner, repo, 1, api)
    nr_pages = int(first_commit_pg.headers['X-Pagecount'])
    commits = []
    commits = commits + first_commit_pg_list
    if nr_pages > 1:
        for i in range(2, nr_pages+1):
            commits_page, commits_page_list = get_commits_per_page(owner, repo, i, api)
            commits = commits + commits_page_list
    return commits

time: 6.99 ms


In [180]:
def commits_of_org(org, api):
    org_repos = repos_of_org(org)
    commits = []
    for index, row in org_repos.iterrows():
        commits = commits + commits_of_repo(row['name'],row['owner.login'], api)
    return commits

time: 1.98 ms


In [225]:
org_commits = commits_of_org('Huknow', api)

time: 1min 55s


In [227]:
commits_df = json_normalize(org_commits)

time: 1.73 s


In [228]:
del commits_df['parents']

time: 28 ms


In [229]:
commits_df['commit_date'] =  pd.to_datetime(commits_df['commit.committer.date'])

time: 1.27 s


In [230]:
commits_df['commit_date'] =  pd.to_datetime(commits_df['commit_date'], utc=True)

time: 37 ms


In [234]:
commits_df['commit_hour'] = commits_df['commit_date'].dt.hour

time: 4.02 ms


In [235]:
commits_df.to_sql(con=engine, name='commits',
                 if_exists='replace', dtype=objects_to_strings(commits_df))

time: 8.37 s


# Github API

In [5]:
github_api = "https://api.github.com"
gh_session = requests.Session()
gh_session.auth = (config.GITHUB_USERNAME, config.GITHUB_TOKEN)

time: 1.98 ms


## 1. Branches

In [18]:
def branches_of_repo(repo, owner, api):
    branches = []
    next = True
    i = 1
    while next == True:
        url = api + '/repos/{}/{}/branches?page={}&per_page=100'.format(owner, repo, i)
        branch_pg = gh_session.get(url = url)
        branch_pg_list = [dict(item, **{'repo_name':'{}'.format(repo)}) for item in branch_pg.json()]    
        branch_pg_list = [dict(item, **{'owner':'{}'.format(owner)}) for item in branch_pg_list]
        branches = branches + branch_pg_list
        if 'rel="next"' not in branch_pg.headers['Link']:
            next = False
        i = i + 1
    return branches

time: 6 ms


In [7]:
superset_branches = json_normalize(branches_of_repo('incubator-superset', 'apache', github_api))

time: 872 ms


In [ ]:
## store permanently into an SQL database
superset_branches.to_sql(con=engine, name='superset_branches',
                 if_exists='replace', dtype=objects_to_strings(superset_branches))

In [9]:
superset_branches.to_csv('data/branches.csv')

time: 24 ms


## 2. Commits

In [17]:
def commits_of_repo_github(repo, owner, api):
    commits = []
    next = True
    i = 1
    while next == True:
        url = api + '/repos/{}/{}/commits?page={}&per_page=100'.format(owner, repo, i)
        commit_pg = gh_session.get(url = url)
        commit_pg_list = [dict(item, **{'repo_name':'{}'.format(repo)}) for item in commit_pg.json()]    
        commit_pg_list = [dict(item, **{'owner':'{}'.format(owner)}) for item in commit_pg_list]
        commits = commits + commit_pg_list
        if 'rel="next"' not in commit_pg.headers['Link']:
            next = False
        i = i + 1
    return commits

time: 6 ms


In [11]:
def create_commits_df(repo, owner, api):
    commits_list = commits_of_repo_github(repo, owner, api)
    commits_df = json_normalize(commits_list)
    del commits_df['parents']
    commits_df['commit_date'] =  pd.to_datetime(commits_df['commit.committer.date'])
    commits_df['commit_date'] =  pd.to_datetime(commits_df['commit_date'], utc=True)
    commits_df['commit_hour'] = commits_df['commit_date'].dt.hour
    return commits_df

time: 3.96 ms


In [14]:
superset_df = create_commits_df('incubator-superset', 'apache', github_api)

time: 33.5 s


In [ ]:
## store permanently into an SQL database
superset_df.to_sql(con=engine, name='superset_commits',
                 if_exists='replace', dtype=objects_to_strings(superset_df))

In [46]:
superset_df.to_csv('data/commits.csv')

time: 378 ms


## 3. Pull Requests

In [15]:
def pulls_of_repo(repo, owner, api):
    pulls = []
    next = True
    i = 1
    while next == True:
        url = api + '/repos/{}/{}/pulls?page={}&per_page=100'.format(owner, repo, i)
        pull_pg = gh_session.get(url = url)
        pull_pg_list = [dict(item, **{'repo_name':'{}'.format(repo)}) for item in pull_pg.json()]    
        pull_pg_list = [dict(item, **{'owner':'{}'.format(owner)}) for item in pull_pg_list]
        pulls = pulls + pull_pg_list
        if 'Link' in pull_pg.headers:
            if 'rel="next"' not in pull_pg.headers['Link']:
                next = False
        i = i + 1
    return pulls

time: 1.99 ms


In [91]:
superset_pulls = json_normalize(pulls_of_repo('incubator-superset', 'apache', github_api))

time: 2.69 s


In [ ]:
## store permanently into an SQL database
superset_pulls.to_sql(con=engine, name='superset_pulls',
                 if_exists='replace', dtype=objects_to_strings(superset_pulls))

In [93]:
superset_pulls.to_csv('data/pulls.csv')

time: 30 ms


## 4. Issues

In [16]:
def issues_of_repo(repo, owner, api):
    issues = []
    next = True
    i = 1
    while next == True:
        url = api + '/repos/{}/{}/issues?page={}&per_page=100'.format(owner, repo, i)
        issue_pg = gh_session.get(url = url)
        issue_pg_list = [dict(item, **{'repo_name':'{}'.format(repo)}) for item in issue_pg.json()]    
        issue_pg_list = [dict(item, **{'owner':'{}'.format(owner)}) for item in issue_pg_list]
        issues = issues + issue_pg_list
        if 'Link' in issue_pg.headers:
            if 'rel="next"' not in issue_pg.headers['Link']:
                next = False
        i = i + 1
    return issues

time: 4.98 ms


In [95]:
superset_issues = json_normalize(issues_of_repo('incubator-superset', 'apache', github_api))

time: 4.92 s


In [ ]:
## store permanently into an SQL database
superset_issues.to_sql(con=engine, name='superset_issues',
                 if_exists='replace', dtype=objects_to_strings(superset_issues))

In [98]:
superset_issues.to_csv('data/issues.csv')

time: 32 ms


## Generating All Repo Data

In [20]:
def generate_repo_data(repo, owner, api):
    branches = json_normalize(branches_of_repo(repo, owner, api))
    commits = create_commits_df(repo, owner, api)
    pulls = json_normalize(pulls_of_repo(repo, owner, api))
    issues = json_normalize(issues_of_repo(repo, owner, api))
    branches.to_csv('data/branches.csv')
    commits.to_csv('data/commits.csv')
    pulls.to_csv('data/pulls.csv')
    issues.to_csv('data/issues.csv')

time: 1 ms


In [21]:
generate_repo_data('incubator-superset', 'apache', github_api)

time: 37 s


## Contribution Statistics

In [22]:
def statistics_of_repo(repo, owner, api):
    contributors = []
    next = True
    i = 1
    while next == True:
        url = api + '/repos/{}/{}/stats/contributors?page={}&per_page=100'.format(owner, repo, i)
        contrib_pg = gh_session.get(url = url)
        contrib_pg_list = [dict(item, **{'repo_name':'{}'.format(repo)}) for item in contrib_pg.json()]    
        contrib_pg_list = [dict(item, **{'owner':'{}'.format(owner)}) for item in contrib_pg_list]
        contributors = contributors + contrib_pg_list
        if 'Link' in contrib_pg.headers:
            if 'rel="next"' not in contrib_pg.headers['Link']:
                next = False
        i = i + 1
    return contributors

time: 3 ms


In [ ]:
superset_contribs = statistics_of_repo('incubator-superset', 'apache', github_api)

In [ ]:
weeks_list = []
for i in (contrib_list):
    for j in i['weeks']:
        j['author'] = i['author']['login']
weeks_list.append(j)
weeks_df = json_normalize(weeks_list)
weeks_df['date'] = pd.to_datetime(weeks_df['w'],unit='s')
weeks_df['week'] = weeks_df['date'].dt.week

In [89]:
weeks_df.to_sql(con=engine, name='contributions',
                 if_exists='replace', dtype=objects_to_strings(weeks_df))

time: 6 ms


time: 143 ms
